In [22]:
from datasets import load_dataset
import re
import random

In [39]:
patterns = [
        r"Imagine you are (?:an?|the)? ([^.,\n]+)",  # Matches "Imagine you are a/an/the X"
        r"As (?:an?|the)? ([^.,\n]+)",              # Matches "As a/an/the X"
        r"You are (?:an?|the)? ([^.,\n]+)",         # Matches "You are a/an/the X"
        r"Act as (?:an?|the)? ([^.,\n]+)",          # Matches "Act as a/an/the X"
        r"the perspective of (?:an?|the)? ([^.,\n]+)",  # Matches "Take on the role of a/an/the X"
        r"Take on the role of (?:an?|the)? ([^.,\n]+)",  # Matches "Take on the role of a/an/the X"
        r"Respond as (?:an?|the)? ([^.,\n]+)",      # Matches "Respond as a/an/the X"
        r"Answer as (?:an?|the)? ([^.,\n]+)",       # Matches "Answer as a/an/the X"
    ]

match = re.match(patterns[1], "As a mother who is, not great")
match.group(1)

'mother who is'

In [40]:
dataset = load_dataset("allenai/tulu-3-sft-personas-instruction-following")["train"]
dataset

Dataset({
    features: ['id', 'prompt', 'messages', 'constraints'],
    num_rows: 29980
})

In [41]:
def find_persona(row):
    persona = None
    for patt in patterns:
        match = re.match(patt, row["prompt"], re.IGNORECASE)
        if match is not None:
            if match.group(1) is not None:
                persona = " ".join(match.group(1).strip().split())
                break
    row['persona'] = persona
    
    return row

In [42]:
dataset = dataset.map(find_persona)
persona_data=dataset.filter(lambda row: row['persona'] is not None)
persona_data

Map:   0%|          | 0/29980 [00:00<?, ? examples/s]

Filter:   0%|          | 0/29980 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'prompt', 'messages', 'constraints', 'persona'],
    num_rows: 2947
})

In [43]:
dataset.select(random.sample(range(len(dataset)), k=5))['prompt']

['Draft a public notice for the financial regulations newsletter, outlining the recent changes in transaction monitoring requirements. Ensure that the text effectively communicates the importance of compliance, but limit the use of capitalized words to no more than 5 throughout the notice.',
 "write a detailed report on the history and impact of the country women's association (cwa) in australia, focusing on its contributions to rural communities and its role in promoting women's rights. include specific examples of initiatives and projects led by the cwa that have benefited rural areas. ensure that all text is in english and lowercase.",
 'Write a mathematical dialogue between two characters, where they explore the concept of compactness in point-set topology. The dialogue should be divided into three sections: Introduction, Exploration, and Conclusion. The total length of the dialogue should be exactly 600 words. End the dialogue with the sentence: "And that\'s why compactness is suc

In [44]:
list(zip(persona_data['persona'], persona_data['prompt']))

[('financial advisor with expertise in the insurance sector',
  'As a financial advisor with expertise in the insurance sector, create an informative guide on "The Evolution of Life Insurance Policies in the United States" for your clients in Brentwood, Tennessee. Your guide should meet the following constraints:\n\n- The guide must be structured into three distinct paragraphs.\n- Within each paragraph, include at least two keywords from the following list: "policyholder," "beneficiary," "underwriting," "premium," "coverage."\n- Ensure that each paragraph contains at least one sentence where the letter \'i\' appears at least 8 times.\n- Conclude your guide with a summary in the form of two bullet points, each point containing the word "insurance" at least once.'),
 ('well-established Hollywood celebrity known for my dance skills in musicals',
  'As a well-established Hollywood celebrity known for my dance skills in musicals, I want to explore new dance trends and innovations in choreog

In [10]:
persona_data.filter(lambda row: row['persona']=="older South Sudanese immigrant who")['prompt']

['Imagine you are an older South Sudanese immigrant who has faced many challenges and now serves as a source of inspiration and support to your community. Create a verifiable instruction that you might ask me to do:\n\n1. Explain the importance of resilience to a group of high school students using no more than 4 sentences.\n2. Provide two responses:\n   - The first response should include a bullet list with at least 3 points.\n   - The second response should be a narrative story of resilience involving a character overcoming a significant challenge.']

These personas are derived from Persona Hub, which is all synthetic, and we end up with examples like the above, which are definitely weird to finetune as a persona? These are way too fine-grained!

## Dolly 15k?

In [45]:
dataset = load_dataset("databricks/databricks-dolly-15k")["train"]
dataset

Dataset({
    features: ['instruction', 'context', 'response', 'category'],
    num_rows: 15011
})

In [46]:
def find_persona2(row):
    persona = None
    for patt in patterns:
        match = re.match(patt, row["instruction"], re.IGNORECASE)
        if match is not None:
            if match.group(1) is not None:
                persona = " ".join(match.group(1).strip().split())
                break
    row['persona'] = str(persona)
    
    return row
    
dataset = dataset.map(find_persona2)

Map:   0%|          | 0/15011 [00:00<?, ? examples/s]

In [47]:
persona_data=dataset.filter(lambda row: row['persona'] != "None")
persona_data

Filter:   0%|          | 0/15011 [00:00<?, ? examples/s]

Dataset({
    features: ['instruction', 'context', 'response', 'category', 'persona'],
    num_rows: 24
})

In [51]:
persona_data['persona']

['golfer',
 'expert',
 'first',
 'firefighter',
 'immigrant',
 'grandparent',
 'young',
 'CEO',
 'last',
 'mom',
 'immortal',
 'java',
 'last',
 'last',
 '16-year',
 'leader',
 'business',
 'child',
 'product',
 'last',
 'Manager',
 'last',
 'last',
 'last']

In [54]:
dataset.select(random.sample(range(len(dataset)), k=5))['instruction']

['How is the climate in Chicago IL',
 'In the walk of fame, which star is on the wall?',
 'What watch should I buy?',
 'Identify which instrument is string or percussion: Riq, Surbahar',
 'Why are dogs so good at smelling?']